In [25]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.cohort.cohort import CohortProcessor
from utils.ethogram import ethogram

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/database.xlsx'
# fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                     Slp filename Start time
0           0       3     Day  2020_08_23_16_00_38_413695_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_16_00_38_413695_compressed_Day.slp   16:00:38
1           1       3     Day  2020_08_23_17_06_55_680620_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_17_06_55_680620_compressed_Day.slp   17:06:55
2           2       3     Day  2020_08_23_18_13_11_710750_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_18_13_11_710750_compressed_Day.slp   18:13:11
3           3       3    Both  2020_08_23_19_19_27_606024_compressed.mp4     P15             6    47:43, day, night        1           1    2020_08_23_19_19_27_606024_compressed_Day.slp   19:19:27
4         

In [31]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################

# NOTE The hybrid video processing seems to break when using parallel flag

#
cohort.n_cores = 30
cohort.parallel = True
cohort.interpolate_flag = True
cohort.exclude_huddles = False   # Don't use this here yet; 
                                 # NEED TO FIRST FIX THE HUDDLES THEN RUN THIS
                                 # See cell 3 below which does this after
cohort.fps = 24
cohort.preprocess_feature_tracks()

#
print ("DONE...")

Found # of Day/night slp files:  300


100%|█████████████████████████████████████████████| 300/300 [00:00<00:00, 46453.69it/s]

Found # of both slp files:  104



100%|██████████████████████████████████████████████| 104/104 [00:00<00:00, 4668.87it/s]


Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_23_19_19_27_606024_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_23_19_19_27_606024_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_26_08_05_30_493031_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_26_08_05_30_493031_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_27_07_18_45_501143_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_27_07_18_45_501143_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_27_19_55_02_593991_compressed_Day.slp
Missing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_27_19_55_02_593991_compressed_Day.slp
Missing:  /mnt/b3a68699-

In [32]:
#########################################################
############## PREPROCESS HUDDLES #######################
#########################################################

# NOTE The hybrid video processing seems to break when using parallel flag

# 
cohort.n_cores = 30 
cohort.parallel = False 
cohort.recompute_flag = False 

# track fix parms
cohort.fix_track_flag = True       
cohort.max_jump_allowed = 50   # maximum distance that a gerbil/huddle can move in 1 frame
cohort.max_dist_to_join = 50   # maximum distnace between 2 chunks that can safely be merged
cohort.min_chunk_len = 24      # shortest duration of huddle; everything else gets deleted

# interploation params
cohort.interpolate_flag = True 
cohort.max_distance_huddle = 50           # how far away we can merge huddles together (pixels)
cohort.max_time_to_join_huddle = 5*24    # how far in time can we merge huddle chunks (seconds x frames)
cohort.min_huddle_time = 24            # minimum huddle duration in frames
              
#
cohort.fps = 24
cohort.preprocess_huddle_tracks()

# 
print ("DONE...")



# of huddle day/night huddle files found:  296


100%|████████████████████████████████████████████| 296/296 [00:00<00:00, 105490.18it/s]


# of huddle hybrid huddle files found:  104


100%|█████████████████████████████████████████████| 104/104 [00:00<00:00, 75014.21it/s]

light_switch time:  47:43 68712 ['day', 'night']


light_switch time:  47:43 68712 ['day', 'night']
light_switch time:  38:50 55920 ['night', 'day']
light_switch time:  38:50 55920 ['night', 'day']
light_switch time:  29:55 43080 ['day', 'night']
light_switch time:  29:55 43080 ['day', 'night']
light_switch time:  20:33 29592 ['night', 'day']
light_switch time:  20:33 29592 ['night', 'day']
light_switch time:  11:21 16344 ['day', 'night']
light_switch time:  11:21 16344 ['day', 'night']
light_switch time:  1:42 2448 ['night', 'day']
light_switch time:  1:42 2448 ['night', 'day']
light_switch time:  48:27 69768 ['night', 'day']
light_switch time:  48:27 69768 ['night', 'day']
light_switch time:  12:10 17520 ['day', 'night']
light_switch time:  12:10 17520 ['day', 'night']
light_switch time:  6:43 9672 ['night', 'day']
light_switch time:  6:43 9672 ['night', 'day']
light_switch time:  1:22 1968 ['day', 'night']
light_switch time:  1:22 1968 ['day', 'night']
light_switch time:  59:16 85344 ['day', 'night']
light_switch time:  59:16 85344 

In [34]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
cohort.huddle_min_distance = 200    # min distance from huddle in pixels for feature to be kept

# this uses the _spine.npy data for tracks; and _huddle_spine_fixed_interpolated.npy for huddles
# if we want to use something else, we can also code it
cohort.parallel = True
cohort.n_cores = 30
cohort.remove_huddles_from_feature_tracks()

#


# file pairs found:  342  (if less  than above, please check missing)


 12%|██████                                           | 42/342 [02:48<17:15,  3.45s/it]

missing: /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/huddles/2020_08_26_13_37_04_739828_compressed_Day_huddle_spine_fixed_interpolated.npy /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_26_13_37_04_739828_compressed_Day_spine.npy


 39%|██████████████████▉                             | 135/342 [06:08<12:14,  3.55s/it]

missing: /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/huddles/2020_08_27_17_43_13_620087_compressed_Day_huddle_spine_fixed_interpolated.npy /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_08_27_17_43_13_620087_compressed_Day_spine.npy


 64%|██████████████████████████████▋                 | 219/342 [09:40<04:01,  1.96s/it]

missing: /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/huddles/2020_09_05_04_08_31_474256_compressed_Night_huddle_spine_fixed_interpolated.npy /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/features/2020_09_05_04_08_31_474256_compressed_Night_spine.npy


100%|████████████████████████████████████████████████| 342/342 [15:01<00:00,  2.64s/it]


UnboundLocalError: local variable 'huddles' referenced before assignment